In [2]:
from core.fractal_framework import FractalFramework, FractalTask
from agents.chatbot_agent import ChatbotAgent, LLMProvider, TaskType
from core.fractal_context import FractalContext
from utils.vector_store import LocalVectorStore
import asyncio
from typing import List, Dict, Any
from dotenv import load_dotenv
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


In [6]:
load_dotenv(dotenv_path='.env')

 # Configure LLM with your API key
llm_config = {
    'provider': 'openai',
    'api_key': os.getenv('OPENAI_API_KEY')
}

# Create framework
framework = FractalFramework()

# Create vector store instance
vector_store = LocalVectorStore(
    api_key=llm_config['api_key'],  # Use the same API key
    model_name="text-embedding-3-small",
    auto_save=True,
    save_path="code_vectors/store"
)

TypeError: LocalVectorStore.__init__() got an unexpected keyword argument 'auto_save'

In [ ]:
code_data = [
    {
        'id': 'file-_app.js',
        'data': {
            'code': '''const login = () => {
                const [email, setEmail] = useState();
                const [password, setPassword] = useState();
                const handleSubmit = () => {
                    console.log(email, password);
                };
                return (
                    <div>
                        <input type="text" value={email} onChange={(e) => setEmail(e.target.value)} />
                        <input type="password" value={password} onChange={(e) => setPassword(e.target.value)} />
                        <button onClick={handleSubmit}>Login</button>
                    </div>
                );
            }''',
            'filePath': '/path/to/file',
            'label': '_app.js'
        }
    },
    {
        'id': 'file-_product.js',
        'data': {
            'code': '''class Product {
                constructor(name, price) {
                    this.name = name;
                    this.price = price;
                }
                toString() {
                    return `${this.name} - $${this.price}`;
                }
            }''',
            'filePath': '/path/to/file',
            'label': '_product.js'
        }
    },
    {
        'id': 'file-_order.js',
        'data': {
            'code': '''class Order {
                constructor(orderId, productList) {
                    this.orderId = orderId;
                    this.productList = productList;
                }
                getTotal() {
                    return this.productList.reduce((total, product) => total + product.price, 0);
                }
            }''',
            'filePath': '/path/to/file',
            'label': '_order.js'
        }
    },
    {
        'id': 'file-_user.js',
        'data': {
            'code': '''class User {
                constructor(username, email) {
                    this.username = username;
                    this.email = email;
                }
                getDetails() {
                    return `${this.username} (${this.email})`;
                }
            }''',
            'filePath': '/path/to/file',
            'label': '_user.js'
        }
    }
]

In [ ]:
for data in code_data:
        if 'data' in data and 'code' in data['data']:
            code = data['data']['code']
            if code:  # Only index non-empty code
                try:
                    # Add to vector store with metadata
                    metadata = {
                        'filePath': data['data'].get('filePath', ''),
                        'label': data['data'].get('label', ''),
                        'id': data['id'],
                        'code': code  # Include original code
                    }
                    vector_store.add_embedding(data['id'], code, metadata)
                    print(f"Indexed: {data['data'].get('label', 'unnamed')}")
                except Exception as e:
                    print(f"Error indexing code: {e}")


In [ ]:
# Since code_data is a list, we can convert it to an iterator
stats = vector_store.bulk_index_code(iter(code_data), batch_size=2)

# Print statistics
print("Indexing Statistics:", stats)